In [1]:
import pandas as pd
import os
import shutil
import numpy as np

from QC.utils import shell_do

In [2]:
basedir = '/data/CARD/PD/GP2/raw_genotypes'
out_genotypes = '/data/CARD/PD/GP2/genotypes'
cohort_path = f'{basedir}/coriell'
sample_info_path = f'{cohort_path}/sample_info'
ilmn_files_path = '/data/CARD/PD/GP2/ilmn_files'
bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A1.bpm'
egt = f'{ilmn_files_path}/NBSCluster_file_n1393_011921.egt'
key_file = f'{sample_info_path}/KEY_FILE_GP2_Coriell_P1_25_042621_B1.txt'
pheno_file = f'{sample_info_path}/CORIELL_20210201_clinical.csv'

ped_dir = f'{cohort_path}/ped'
plink_dir = f'{cohort_path}/plink'
swarm_scripts_dir = f'{cohort_path}/swarm_scripts'
idat_dir = f'{cohort_path}/idats'
raw_idat_file_path = f'{cohort_path}/idats/GP2_coriell_idats'

# create ped and plink directories for raw geno outputs if they don't exist
os.makedirs(ped_dir, exist_ok=True)
os.makedirs(plink_dir, exist_ok=True)
os.makedirs(f'{plink_dir}/indiv_samples', exist_ok=True)
os.makedirs(swarm_scripts_dir, exist_ok=True)
os.makedirs(idat_dir, exist_ok=True)

iaap = f'{ilmn_files_path}/iaap-cli/iaap-cli'

In [3]:
pheno = pd.read_csv(pheno_file)
key = pd.read_csv(key_file, sep='\t')


pheno['Original_clinicalID'] = pheno['Original_clinicalID'].astype(str)
key['Sample_ID'] = key['Sample_ID'].astype(str)
key['filename'] = key['SentrixBarcode_A'].astype(str) + '_' + key['SentrixPosition_A']
pheno_out = key.merge(pheno, how='inner', left_on='Sample_ID', right_on='Original_clinicalID')
pheno_out['IID'] = pheno_out.SentrixBarcode_A.astype(str) + '_' + pheno_out.SentrixPosition_A.astype(str)
pheno_out['FID'] = 0
pheno_out['FID_new'] = 0
pheno_out['pheno'] = 0
pheno_out.loc[pheno_out.Phenotype == 'PD', 'pheno'] = 2
pheno_out.loc[pheno_out.Phenotype == 'Control', 'pheno'] = 1
pheno_out.loc[pheno_out.Phenotype == np.nan, 'pheno'] = 0

pheno_out[['FID','IID', 'FID_new', 'Sample_ID']].to_csv(f'{sample_info_path}/update_ids.txt', sep='\t', header=False, index=False)
pheno_out[['FID_new', 'Sample_ID', 'pheno']].to_csv(f'{sample_info_path}/update_pheno.txt', sep='\t', header=False, index=False)

In [4]:
# create a folder in idats for each plate in new idat_dir
for code in key.SentrixBarcode_A.unique():
    os.makedirs(f'{idat_dir}/{code}', exist_ok=True)

# copy idat intensity files to respective directories under idat_dir
missing_idats = []

for i, filename in enumerate(key.filename):
    sentrix_code = key.SentrixBarcode_A.iloc[i]
    grn = f'{raw_idat_file_path}/{filename}_Grn.idat'
    red = f'{raw_idat_file_path}/{filename}_Red.idat'

    if os.path.isfile(grn):
        shutil.copyfile(src=grn, dst=f'{idat_dir}/{sentrix_code}/{filename}_Grn.idat')
    else:
        missing_idats.append(grn)

    if os.path.isfile(red):
        shutil.copyfile(src=red, dst=f'{idat_dir}/{sentrix_code}/{filename}_Red.idat')
    else:
        missing_idats.append(red)

len(missing_idats)

KeyboardInterrupt: 

In [ ]:
with open(f'{swarm_scripts_dir}/idat_to_ped.swarm', 'w') as f:
    
    for code in key.SentrixBarcode_A.unique():
        
        idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{ped_dir}/ \
-f {idat_dir}/{code} \
-p \
-t 8'
        
        f.write(f'{idat_to_ped_cmd}\n')
f.close()


In [ ]:
!swarm -f {swarm_scripts_dir}/idat_to_ped.swarm -g 32 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

In [58]:
# copy map file to match name of each ped
map_file = f'{ped_dir}/NeuroBooster_20042459_A1.map'
for filename in key.filename:
    ped = f'{ped_dir}/{filename}.ped'
    out_map = f'{ped_dir}/{filename}.map'
    if os.path.isfile(ped):
        shutil.copyfile(src=map_file, dst=out_map)
    else:
        print(f'{ped} does not exist!')
        print(f'{out_map} creation cancelled')

In [59]:

with open(f'{swarm_scripts_dir}/make_bed.swarm', 'w') as f:
    for filename in key.filename:
        ped = f'{ped_dir}/{filename}'
        make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {plink_dir}/indiv_samples/{filename}'

        f.write(f'{make_bed_cmd}\n')
f.close()


In [60]:
!swarm -f {swarm_scripts_dir}/make_bed.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

14634832


In [61]:
# write plink merge command
with open(f"{plink_dir}/merge_bed.list", 'w') as f:
    for filename in key.filename:
        bed = f'{plink_dir}/indiv_samples/{filename}'
        f.write(f'{bed}\n')
f.close()

with open(f"{swarm_scripts_dir}/merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list {plink_dir}/merge_bed.list \
--update-ids {sample_info_path}/update_ids.txt \
--make-bed \
--out {plink_dir}/indiv_samples/coriell_merge'
    f.write(f"{plink_merge_cmd}")
f.close()

In [63]:
!swarm -f {swarm_scripts_dir}/merge.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

14638754


In [64]:
!plink --bfile {plink_dir}/indiv_samples/coriell_merge --pheno {sample_info_path}/update_pheno.txt --make-bed --out {plink_dir}/coriell

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell.log.
Options in effect:
  --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/indiv_samples/coriell_merge
  --make-bed
  --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell
  --pheno /data/CARD/PD/GP2/raw_genotypes/coriell/sample_info/update_pheno.txt

1547809 MB RAM detected; reserving 773904 MB for main workspace.
2004347 variants loaded from .bim file.
2304 people (1307 males, 941 females, 56 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
/data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell.nosex .
2304 phenotype values present after --pheno.
phenotypes to be ignored, use the --allow-no-sex flag.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2304 founders and 0 nonfounders present.
Calculating allele fre

In [8]:
# run QC pipeline
geno_path = f'{basedir}/coriell/plink/coriell'
out_path = f'{out_genotypes}/coriell/clean/coriell'
os.makedirs(f'{out_genotypes}/coriell/clean/', exist_ok=True)
ref_dir_path = '/data/LNG/vitaled2/1kgenomes'
ref_panel = f'{ref_dir_path}/1kg_ashkj_ref_panel_gp2_pruned'
ref_labels = f'{ref_dir_path}/ref_panel_ancestry.txt'


with open(f'{swarm_scripts_dir}/run_qc_pipeline.swarm','w') as f:
    run_pipeline = f'python3 ../run_qc_pipeline.py --geno {geno_path} --ref {ref_panel} --ref_labels {ref_labels} --out {out_path}'
    f.write(f'{run_pipeline}\n')
f.close()
!cat {swarm_scripts_dir}/run_qc_pipeline.swarm
# shell_do(run_pipeline, log=True, return_log=True)
# !python3 ../run_qc_pipeline.py --geno {geno_path} --ref {ref_panel} --ref_labels {ref_labels} --out {out_path}

python3 ../run_qc_pipeline.py --geno /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell --ref /data/LNG/vitaled2/1kgenomes/1kg_ashkj_ref_panel_gp2_pruned --ref_labels /data/LNG/vitaled2/1kgenomes/ref_panel_ancestry.txt --out /data/CARD/PD/GP2/genotypes/coriell/clean/coriell


In [9]:
!swarm -f {swarm_scripts_dir}/run_qc_pipeline.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

16859681


In [7]:
!cat swarm/swarm_16858135_0.e

import: unable to open X server `' @ error/import.c/ImportImageCommand/369.
import: unable to open X server `' @ error/import.c/ImportImageCommand/369.
../run_qc_pipeline.py: line 5: from: command not found
../run_qc_pipeline.py: line 6: from: command not found
../run_qc_pipeline.py: line 7: from: command not found
../run_qc_pipeline.py: line 10: syntax error near unexpected token `('
../run_qc_pipeline.py: line 10: `parser = argparse.ArgumentParser(description='Arguments for Genotyping QC (data in Plink .bim/.bam/.fam format)')'


In [3]:
# run QC pipeline step-by-step
import pandas as pd
import os
import shutil
import zipfile

from QC.qc import callrate_prune, het_prune, sex_prune, related_prune, variant_prune, avg_miss_rates
from Ancestry.ancestry import run_ancestry, split_cohort_ancestry
from QC.utils import shell_do


In [4]:
geno_path = f'{basedir}/coriell/plink/coriell'
out_path = f'{out_genotypes}/coriell/clean/coriell'
os.makedirs(f'{out_genotypes}/coriell/clean/', exist_ok=True)

In [5]:
avg_miss = avg_miss_rates(geno_path, f'{geno_path}_missing')
avg_miss

callrate_out = f'{geno_path}_callrate'
callrate = callrate_prune(geno_path, callrate_out)

sex_out = f'{callrate_out}_sex'
sex = sex_prune(callrate_out, sex_out)

Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell --missing --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_missing


Average Missing Call Rate (lmiss): 0.01138650122141718
Average Missing Genotyping Rate (imiss): 0.011392406684027762

RUNNING: callrate_prune



Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell --mind 0.02 --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate



RUNNING: sex_prune



Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate --check-sex 0.25 0.75 --maf 0.05 --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_tmp1
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate --chr 23 --from-bp 2699520 --to-bp 154931043 --maf 0.05 --geno 0.05 --hwe 1E-5 --check-sex 0.25 0.75 --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_tmp2
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate --remove /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex.fails --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex


In [6]:
ancestry_out = f'{sex_out}_ancestry'
ref_dir_path = '/data/LNG/vitaled2/1kgenomes'
ref_panel = f'{ref_dir_path}/1kg_ashkj_ref_panel_gp2_pruned'
ref_labels = f'{ref_dir_path}/ref_panel_ancestry.txt'
# ancestry = run_ancestry(geno_path=sex_out, out_path=ancestry_out, ref_panel=ref_panel, ref_labels=ref_labels)

# rerunning with best params
best_params = {'svc__C': [0.01], 'umap__a': [1.0], 'umap__b': [0.25], 'umap__n_components': [15], 'umap__n_neighbors': [20]}
ancestry = run_ancestry(geno_path=sex_out, out_path=ancestry_out, ref_panel=ref_panel, ref_labels=ref_labels, train_param_grid=best_params)


RUNNING: predict_ancestry


RUNNING: calculate_pcs



Executing: plink --bfile /data/LNG/vitaled2/1kgenomes/1kg_ashkj_ref_panel_gp2_pruned --extract /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.common_snps --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps
Executing: flashpca --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps -d 50 --outpc /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.pcs --outvec /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.vec --outval /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.val --outpve /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.pve --outload /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.loadings --outmeansd /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.meansd




Labeled Reference Ancestry Counts:
AFR    504
EAS    504
SAS    489
AJ     471
EUR    404
AMR    347
AAC    157
FIN     99
Name: label, dtype: int64




Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex --extract /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.common_snps --reference-allele /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.ref_allele --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/pcs_common_snps


/data/CARD/PD/GP2/raw_genotypes/coriell/plink/pcs_common_snps /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.meansd /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.loadings /data/CARD/PD/GP2/raw_genotypes/coriell/plink/pcs_common_snps.projections


Executing: flashpca --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/pcs_common_snps --project --inmeansd /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.meansd --inload /data/CARD/PD/GP2/raw_genotypes/coriell/plink/ref_common_snps.loadings --outproj /data/CARD/PD/GP2/raw_genotypes/coriell/plink/pcs_common_snps.projections -v



RUNNING: munge_pca_loadings


RUNNING: train_umap_classifier

Training Balanced Accuracy: 0.9655869259194596
Best Parameters: {'svc__C': 0.01, 'umap__a': 1.0, 'umap__b': 0.25, 'umap__n_components': 15, 'umap__n_neighbors': 20}
Balanced Accuracy on Test Set: 0.984873949579832

predicted:
 EUR    1740
AJ      262
AAC      94
AMR      91
EAS      14
SAS      14
FIN       1
Name: label, dtype: int64



In [7]:
pred_labels_path = ancestry['output']['predicted_labels']['labels_outpath']
cohort_split = split_cohort_ancestry(geno_path=sex_out, labels_path=pred_labels_path, out_path=ancestry_out)

Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex --keep /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ.samples --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex --keep /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_EUR.samples --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_EUR
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex --keep /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_EAS.samples --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_EAS
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex --keep /data/CARD/PD/GP2/raw_genotypes/coriell/plink/corie

In [8]:
het_dict = dict()
related_dict = dict()
variant_dict = dict()

for geno, label in zip(cohort_split['paths'], cohort_split['labels']):

    # related
    related_out = f'{geno}_related'
    related = related_prune(geno, related_out)
    related_dict[label] = related
    
    # het
    het_out = f'{related_out}_het'
    het = het_prune(geno, het_out)
    het_dict[label] = het
    
    # variant
    variant_out = f'{het_out}_variant'
    if het['pass']:
        variant = variant_prune(het_out, variant_out)
        variant_dict[label] = variant
    else:
        variant = variant_prune(related_out, variant_out)
        variant_dict[label] = variant




RUNNING: related_prune



Executing: gcta --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ --autosome --maf 0.05 --make-grm --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_total_grm_tmp
Executing: gcta --grm /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_total_grm_tmp --grm-cutoff 0.125 --make-grm --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_unrelated_grm_tmp
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ --keep /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_unrelated_grm_tmp.grm.id --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related
Executing: gcta --grm /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_total_grm_tmp --grm-cutoff 0.95 --make-grm --out /data/CA


RUNNING: het_prune



Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ --geno 0.01 --maf 0.05 --indep-pairwise 50 5 0.5 --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_het_tmp
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ --extract /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_het_tmp.prune.in --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_het_tmp2
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_het_tmp2 --het --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ_related_het_tmp3
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AJ --remove /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancest


RUNNING: variant_prune



...


RUNNING: het_prune



Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN --extract /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related_het_tmp.prune.in --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related_het_tmp2
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related_het_tmp2 --het --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related_het_tmp3


Heterozygosity pruning failed!
Check /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related_het_tmp.log, /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related_het_tmp2.log, or /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related_het_tmp3.log for more information

RUNNING: variant_prune



Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related --geno 0.05 --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_FIN_related_het_variant_geno_tmp1



Case/control pruning failed! May be missing Controls!
Check number of Cases/Controls



In [9]:
ancestry_counts_df = pd.DataFrame(ancestry['metrics']['predicted_counts']).reset_index()
ancestry_counts_df.columns = ['label', 'count']
ancestry_counts_df

,label,count
0,EUR,1740
1,AJ,262
2,AAC,94
3,AMR,91
4,EAS,14
5,SAS,14
6,FIN,1


In [10]:
# build report
steps = [callrate, sex]
steps2 = [het_dict, related_dict, variant_dict]
metrics_df = pd.DataFrame()

for item in steps:
    
    step = item['step']
    pf = item['pass']
    level = 'sample'
    ancestry_label = 'all'
    
    for metric, value in item['metrics'].items():
        tmp_metrics_df = pd.DataFrame({'step':[step], 'pruned_count':[value], 'metric':[metric], 'ancestry':[ancestry_label], 'level':[level], 'pass': [pf]})
        metrics_df = metrics_df.append(tmp_metrics_df)


for item in steps2:
    for ancestry_label, metrics in item.items():
        
        step = metrics['step']
        pf = metrics['pass']
        
        if step in ['het_prune','related_prune']:
            level = 'sample'
        else:
            level = 'variant'

        for metric, value in metrics['metrics'].items():
            tmp_metrics_df = pd.DataFrame({'step':[step], 'pruned_count':[value], 'metric':[metric], 'ancestry':[ancestry_label], 'level':[level], 'pass': [pf]})
            metrics_df = metrics_df.append(tmp_metrics_df)

metrics_df.reset_index(drop=True, inplace=True)



In [11]:
# build output hdf
metrics_outfile = f'{out_path}.QC.metrics.h5'

le = ancestry['data']['label_encoder']
confusion_matrix = ancestry['data']['confusion_matrix']
conf_mat_df = pd.DataFrame(confusion_matrix)
conf_mat_df.columns = le.inverse_transform([i for i in range(8)])
conf_mat_df.index = le.inverse_transform([i for i in range(8)])

ref_pcs = ancestry['data']['ref_pcs']
projected_pcs = ancestry['data']['projected_pcs']
total_umap = ancestry['data']['total_umap']
ref_umap = ancestry['data']['ref_umap']
new_samples_umap = ancestry['data']['new_samples_umap']
pred_ancestry_labels = ancestry['data']['predict_data']['ids']

metrics_df.to_hdf(metrics_outfile, key='QC', mode='w')
ancestry_counts_df.to_hdf(metrics_outfile, key='ancestry_counts')
pred_ancestry_labels.to_hdf(metrics_outfile, key='ancestry_labels')
conf_mat_df.to_hdf(metrics_outfile, key='confusion_matrix', index=True)
ref_pcs.to_hdf(metrics_outfile, key='ref_pcs')
projected_pcs.to_hdf(metrics_outfile, key='projected_pcs')
total_umap.to_hdf(metrics_outfile, key='total_umap')
ref_umap.to_hdf(metrics_outfile, key='ref_umap')
new_samples_umap.to_hdf(metrics_outfile, key='new_samples_umap')

/home/vitaled2/.local/lib/python3.7/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  f(store)
/home/vitaled2/.local/lib/python3.7/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_items] [items->None]

  f(store)


In [29]:
test_hdf = pd.read_hdf(metrics_outfile, key='ancestry_labels')

In [26]:
# move output to output dir
for label, data in variant_dict.items():
    if data['pass']:
        for suffix in ['bed','bim','fam','hh','log']:
            plink_file = f"{data['output']['plink_out']}.{suffix}"
            plink_outfile = f'{out_path}_{label}.{suffix}'
            shutil.copyfile(src=plink_file, dst=plink_outfile)

In [27]:
out_path

'/data/CARD/PD/GP2/genotypes/coriell/clean/coriell'

In [19]:
# merge AAC with ref panel 

from QC.utils import merge_genos

aac_geno = '/data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant'
out_dir = '/data/CARD/PD/GP2/genotypes/coriell/clean'
ref_dir_path = '/data/LNG/vitaled2/1kgenomes'
ref_panel = f'{ref_dir_path}/1kg_ashkj_ref_panel_gp2_pruned'

merged_ref_geno = f'{aac_geno}_merged_ref'



In [12]:
geno_ancestry_prune1 = f'{aac_geno}_ancestry_prune1'
geno_ancestry_prune2 = f'{aac_geno}_ancestry_prune2'
# geno_ancestry_prune3 = f'{geno_het}_ancestry_prune3'

# prune geno_het for geno, maf, hwe, and palindromes
bim = pd.read_csv(f'{aac_geno}.bim', sep='\t', header=None)

# find and drop palindromes in geno_het bim file
bim.columns = ['chr', 'rsid', 'kb', 'pos', 'a1', 'a2']
palindromes = bim.loc[((bim.a1 == 'A') & (bim.a2 == 'T')) | ((bim.a1 == 'T') & (bim.a2 == 'A')) | ((bim.a1 == 'C') & (bim.a2 == 'G')) | ((bim.a1 == 'G') & (bim.a2 == 'C'))]
palindromes['rsid'].to_csv(f'{aac_geno}_palindromes.snplist', header=False, index=False, sep='\t')

plink_cmd1 = f'plink --bfile {aac_geno}\
 --maf 0.05\
 --geno 0.01\
 --hwe 0.0001\
 --autosome\
 --allow-no-sex\
 --exclude {aac_geno}_palindromes.snplist\
 --make-bed\
 --out {geno_ancestry_prune1}' 

#remove high-LD regions
plink_cmd2 = f'plink --bfile {geno_ancestry_prune1}\
 --exclude range {ref_dir_path}/hg19_exclusion_regions.txt\
 --autosome\
 --allow-no-sex\
 --make-bed\
 --out {geno_ancestry_prune2}'

cmds = [plink_cmd1, plink_cmd2]

for cmd in cmds:
    shell_do(cmd)

Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant --maf 0.05 --geno 0.01 --hwe 0.0001 --autosome --allow-no-sex --exclude /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_palindromes.snplist --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune1
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune1 --exclude range /data/LNG/vitaled2/1kgenomes/hg19_exclusion_regions.txt --autosome --allow-no-sex --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2


In [32]:
# now get common snps between pruned ref panel and pruned geno
geno_pruned_bim = pd.read_csv(f'{geno_ancestry_prune2}.bim', sep='\t', header=None)
geno_pruned_bim.columns = ['chr', 'rsid', 'kb', 'pos', 'a1', 'a2']
ref_bim = pd.read_csv(f'{ref_panel}.bim', sep='\t', header=None)
ref_bim.columns = ['chr', 'rsid', 'kb', 'pos', 'a1', 'a2']

common_snps = ref_bim.merge(geno_pruned_bim, how='inner', on=['rsid'])

common_snps['rsid'].to_csv(f'{out_dir}/aac_geno_ref_panel_common.snps', sep='\t', header=False, index=False)

In [38]:
ref_panel_common_snps_geno_out = f'{out_dir}/aac_geno_ref_panel_common_snps'
ref_panel_common_snps = f'{out_dir}/aac_geno_ref_panel_common.snps'

ext_snps_cmd = f'plink --bfile {ref_panel} --extract {ref_panel_common_snps} --make-bed --out {ref_panel_common_snps_geno_out}'
shell_do(ext_snps_cmd)

Executing: plink --bfile /data/LNG/vitaled2/1kgenomes/1kg_ashkj_ref_panel_gp2_pruned --extract /data/CARD/PD/GP2/genotypes/coriell/clean/aac_geno_ref_panel_common.snps --make-bed --out /data/CARD/PD/GP2/genotypes/coriell/clean/aac_geno_ref_panel_common_snps


In [40]:
# get reference alleles from ref_panel_common_snps
ref_panel_common_snps_ref_alleles = f'{ref_panel_common_snps_geno_out}.ref_allele'
ref_panel_common_snps_bim = pd.read_csv(f'{ref_panel_common_snps_geno_out}.bim', header=None, sep='\t')
ref_panel_common_snps_bim.columns = ['chr', 'rsid', 'kb', 'pos', 'a1', 'a2']
ref_panel_common_snps_bim[['rsid','a1']].to_csv(ref_panel_common_snps_ref_alleles, sep='\t', header=False, index=False)

In [42]:
geno_common_snps = f'{geno_ancestry_prune2}_common_snps'

ext_snps_cmd = f'plink --bfile {geno_ancestry_prune2} --extract {ref_panel_common_snps} --reference-allele {ref_panel_common_snps_ref_alleles} --make-bed --out {geno_common_snps}'
shell_do(ext_snps_cmd)

Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2 --extract /data/CARD/PD/GP2/genotypes/coriell/clean/aac_geno_ref_panel_common.snps --reference-allele /data/CARD/PD/GP2/genotypes/coriell/clean/aac_geno_ref_panel_common_snps.ref_allele --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2_common_snps


In [45]:
merged_ref_geno = f'{geno_ancestry_prune2}_merged_ref'

merge_genos(geno_common_snps, ref_panel_common_snps_geno_out, merged_ref_geno)

Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2_common_snps --allow-no-sex --bmerge /data/CARD/PD/GP2/genotypes/coriell/clean/aac_geno_ref_panel_common_snps --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2_merged_ref --make-bed
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2_common_snps --allow-no-sex --flip /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2_merged_ref-merge.missnp --make-bed --out /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2_common_snps_flip
Executing: plink --bfile /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell_callrate_sex_ancestry_AAC_related_het_variant_ancestry_prune2_

In [55]:
ancestry = pd.read_csv('/data/LNG/vitaled2/1kgenomes/ref_panel_ancestry.txt', sep='\t', header=None, names=['FID','IID','label'])
ref_fam = pd.read_csv(f'{ref_panel}.fam', sep=' ', header=None)
ref_labeled = ref_fam.merge(ancestry, how='left', left_on=[0,1], right_on=['FID','IID'])

In [76]:
geno_common_snps_df = pd.read_csv(f'{geno_common_snps}.fam', header=None, sep='\s+', usecols=[0,1], names=['FID','IID'])
geno_common_snps_df.loc[:,'label'] = 'new'
fs_labels = geno_common_snps_df.append(ref_labeled.loc[:,['FID','IID','label']])
fs_labels.to_csv(f'{merged_ref_geno}.labels', sep='\t', index=False)

In [77]:

ancestry_labels = f'{merged_ref_geno}.labels'
structure_out = f'{merged_ref_geno}_structure'
# fam = pd.read_csv(f'{ref_panel_prune_final}.fam', sep=' ', header=None)
structure = f'/data/vitaled2/ref_panel/fastStructure/structure.py'


# run for k=1-8 and use chooseK.py to select best
fs_swarm_script = f'{out_dir}/faststructure_choosek.swarm'
with open(fs_swarm_script, 'w') as f:
    k=8
    fs_cmd = f'bash /data/vitaled2/GWAS/gwas/faststructure_setup_and_run.sh -i {merged_ref_geno} -o {structure_out} -f {structure} -k {k}'
    f.write(f'{fs_cmd}\n')
f.close()

In [79]:
shell_do(f'swarm -f {fs_swarm_script} -g 16 --time=10:00:00 -t 20 --logdir {ref_dir_path}/swarm --partition=norm')

Executing: swarm -f /data/CARD/PD/GP2/genotypes/coriell/clean/faststructure_choosek.swarm -g 16 --time=10:00:00 -t 20 --logdir /data/LNG/vitaled2/1kgenomes/swarm --partition=norm


In [81]:
fam = pd.read_csv(f'{merged_ref_geno}.fam', sep=' ', header=None)
pop = pd.read_csv(f'{merged_ref_geno}.labels', sep='\t')
q_df = pd.read_csv(f'{structure_out}.8.meanQ', header=None, sep='\s+')
q_df.columns = [f'pop{i}' for i in range(len(q_df.columns))]

In [87]:
q_df['FID'], q_df['IID'] = fam[0], fam[1]
q_pop_merged = q_df.merge(pop, left_on=['FID','IID'], right_on=['FID','IID'])

In [91]:
q_pop_merged.to_csv(f'{out_dir}/aac_samples_labeled_faststructure.txt', sep='\t', header=True, index=False)

In [93]:
f'{out_dir}/aac_samples_labeled_faststructure.txt'

'/data/CARD/PD/GP2/genotypes/coriell/clean/aac_samples_labeled_faststructure.txt'

0.8523977872340425